In [2]:
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.hashes import SHA256

In [6]:

def generate_aes_key() -> bytes:
    """Генерує випадковий ключ AES.

    Повертає:
        bytes: Згенерований 256-бітний AES-ключ.
    """
    return os.urandom(32)  # Генерується 32 байти випадкових даних (256 біт).


def encrypt_with_aes(key: bytes, plaintext: str) -> tuple[bytes, bytes]:
    """Шифрує текст за допомогою AES у режимі CFB.

    Аргументи:
        key (bytes): AES-ключ для шифрування.
        plaintext (str): Повідомлення, яке потрібно зашифрувати.

    Повертає:
        tuple[bytes, bytes]: Кортеж, що містить вектор ініціалізації (IV) та зашифрований текст.
    """
    iv = os.urandom(16)  # Генерується 16-байтний вектор ініціалізації.
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode('utf-8')) + encryptor.finalize()  # Шифрується повідомлення.

    return iv, ciphertext

Лабораторна робота 1 , Панкєєва(Двоєглазова) СД


In [7]:
def generate_rsa_keypair() -> tuple[rsa.RSAPrivateKey, rsa.RSAPublicKey]:
    """Генерує пару ключів RSA (приватний та публічний).

    Повертає:
        tuple[rsa.RSAPrivateKey, rsa.RSAPublicKey]: Кортеж із приватним та публічним RSA-ключами.
    """
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend())
    public_key = private_key.public_key()

    return private_key, public_key


def sign_data(private_key: rsa.RSAPrivateKey, data: bytes) -> bytes:
    """Підписує дані за допомогою приватного ключа RSA.

    Аргументи:
        private_key (rsa.RSAPrivateKey): Приватний RSA-ключ для підпису.
        data (bytes): Дані для підпису.

    Повертає:
        bytes: Згенерований підпис.
    """
    return private_key.sign(
        data,
        padding.PSS(mgf=padding.MGF1(SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hashes.SHA256(),
    )


In [10]:
def verify_signature(public_key: rsa.RSAPublicKey, signature: bytes, data: bytes) -> bool:
    """Перевіряє RSA-підпис.

    Аргументи:
        public_key (rsa.RSAPublicKey): Публічний ключ RSA для перевірки.
        signature (bytes): Підпис для перевірки.
        data (bytes): Оригінальні дані.

    Повертає:
        bool: True, якщо підпис валідний, False у разі помилки.
    """
    try:
        public_key.verify(
            signature,
            data,
            padding.PSS(mgf=padding.MGF1(SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hashes.SHA256(),
        )
        return True
    except Exception:
        return False  # Якщо перевірка не пройшла, повертається False.


if __name__ == '__main__':
    # Демонстрація шифрування AES
    original_message = 'Конфіденційна інформація'
    aes_key = generate_aes_key()
    iv, ciphertext = encrypt_with_aes(aes_key, original_message)

    print("Лабораторна робота 1 , Панкєєва(Двоєглазова) СД")
    print(f'Оригінальне повідомлення: {original_message}')
    print(f'AES-ключ: {aes_key.hex()}')
    print(f'Вектор ініціалізації (IV): {iv.hex()}')
    print(f'Зашифрований текст: {ciphertext.hex()}')

    # Генерація RSA-ключів та підпис
    private_key, public_key = generate_rsa_keypair()
    data = 'Важливі дані'.encode('utf-8')  # Текст перекодується в байти за допомогою UTF-8
    signature = sign_data(private_key, data)

    print(f'Дані для підпису: {data.decode("utf-8")}')
    print(f'Підпис: {signature.hex()}')

    # Перевірка підпису
    is_valid = verify_signature(public_key, signature, data)
    print(f'Підпис валідний: {is_valid}')

Лабораторна робота 1 , Панкєєва(Двоєглазова) СД
Оригінальне повідомлення: Конфіденційна інформація
AES-ключ: 74d9f46f9c145d73f22fe70c04d838d98d2e8c17e52649e4baac0b558ab739b9
Вектор ініціалізації (IV): f25922bd3a421a669def5388625fb2de
Зашифрований текст: 34513c38388969c9a699f8dd44fbd6b14c3204ef6202ff718a67fc90bf0f4bf17e9eead986030b2e94b04d3d80f716
Дані для підпису: Важливі дані
Підпис: 51749a8148042408360acb949e6cb26621455f0f64a34b1f177d03454bfab3c972973d2dfafbc227519ba03381a16b835715015f79e17f11ca40a3a4463c3ca71a3cf2bbdcb0dd2e7d4f2f695d3a23fd99abb90bc4ba8e26786931229f1d99c6f80d656977b81e1bf54100e6f7bacac3cae8001f4463cb0a6f1907e916d74f6ce499169ee668bd86d90ad3640120cdb5c013c5ddc5c51fc27f6ad1d96dbbba020efe0664c18a486af7b8737f102494fc043d8667cd9689e5e0589c6baffaed04ec8a35fe275cef5eb0793b18d50c83488ae88b7d0219b91253c4997d2de2605f14bd3b1a029fc8f4b30590d1ef37b901c6335e681e49b3baa2036114fa1a1221
Підпис валідний: True
